In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [7]:
autoencoder_model = tf.keras.saving.load_model('autoencoder_model')
classifier = tf.keras.saving.load_model('fake_classifier')
# Create a new model that only includes the encoder part
encoder_model = tf.keras.Model(inputs=autoencoder_model.input, outputs=autoencoder_model.get_layer('dense_6').output)

In [8]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def audio_to_mel_spectrogram(audio_file_path, max_length=4*22500, sr=22500, n_mels=256):
    # Load the audio file
    y, sr = librosa.load(audio_file_path, sr=sr)
    
    if len(y) > max_length:
        # If the audio is longer than the maximum length, cut it
        y = y[:max_length]
    elif len(y) < max_length:
        # If the audio is shorter, pad it with zeros
        pad_length = max_length - len(y)
        y = np.pad(y, (0, pad_length), mode='constant')
    
    # Calculate the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_mels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    return mel_spectrogram

# Generator function for lazy loading of audio data
def audio_data_generator(audio_files):
    for audio_file in audio_files:
        yield audio_to_mel_spectrogram(audio_file).reshape((1, 128, 176, 1))


In [13]:
# Example usage
spectrogram = audio_to_mel_spectrogram("new_audio.m4a", sr=44100).reshape((1, 128, 176, 1))
spectrogram = (spectrogram+80)/80
embedding = encoder_model.predict(spectrogram)
y = classifier.predict(embedding)
y

1/1 [==============================] - 0s 20ms/step


C:\Users\gabri\AppData\Local\Temp\ipykernel_13852\735273640.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file_path, sr=sr)
C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 [==============================] - 0s 16ms/step


array([[0.6073695]], dtype=float32)